# Class

In [1]:
#! /usr/bin/env python
# -*- coding:utf-8 -*-
import serial
import time
import subprocess

from copley_lib.ParamDict_EventStatus import BitsMapped as BitsMapped_ES
from copley_lib.ParamDict_HomingMethod import BitsMapped as BitsMapped_HM
from constant_lib.Constant_Serial import *

class Port():
    '''
        定义Port类
    '''
    defaultPortID = PortID_UCR
    # defaultPortID = PortID_Sting
    # defaultPortID = PortID_Wing
    defaultBaud = 9600
    defaultTimeout = .1 # None

    def __init__(self):
        self.StartSerial()

    def StartSerial(self,baud=defaultBaud,serialPort=defaultPortID,timeout=defaultTimeout):
        '''
            初始化，设置串口通信速率为9600
            超时设置为None就可以，可能是应为这个RS232是全双工的，所以接收和发送的前后脚的
            其他的设置被注释了，应为没用，反而还会有麻烦
        '''
        self.CopleySerial = serial.Serial()
        self.CopleySerial.timeout=timeout
        self.CopleySerial.port=serialPort
    #     self.CopleySerial.baudrate=baud
        # self.CopleySerial.open()
        # self.CopleySerial.write('r \r')
        # time.sleep(5)
    #     self.ChangeSerialSpeed(baud)
    # def ChangeSerialSpeed(self,baud):
    #     # Serial Port Baud Rate. Units: bits/s.
    #     cmd = 's r0x90'+' '+str(baud) + '\r'
    #     self.CopleySerial.write(cmd)
        # self.CopleySerial.close()
        self.CopleySerial.baudrate=9600
        # time.sleep(2)
        self.CopleySerial.open()

    def RxD(self, cmd):
        '''
            调用该函数，写入要传送的ASCII给驱动器
            并使用read_until读取返回的一行数据，以'\r'为终止符
        '''
        cmd = str(self.NodeID) + ' ' + cmd
        self.CopleySerial.write(cmd)
        result = self.CopleySerial.read_until('\r')[0:-1]
        print('RxD: {}\t{}'.format(cmd[:-1], result))
        return result

    def RxD_Line(self, cmd):
        '''
            调用该函数，写入要传送的ASCII给驱动器
            并使用read_until读取返回的一行数据，以'\r'为终止符
        '''
        self.CopleySerial.write(cmd)
        result = self.CopleySerial.read_until('\r')[0:-1]
        print('RxD: {}\t{}'.format(cmd[:-1], result))
        return result

    def PortInfo(self, mode='time_msg'):
        if mode=='full_msg':
            msg = '\n\t- PortID: {}\n\t- Buad: {}\n\t- isOpen: {}\n\t- Readable: {}'.format(self.CopleySerial.portstr, self.CopleySerial.baudrate, self.CopleySerial.isOpen(), self.CopleySerial.readable())
        elif mode=='time_msg':
            msg = '\n\t- PortID: {}\n\t- Time: {}'.format(self.CopleySerial.portstr, time.asctime())
        else:
            msg = '\n\t- PortID: {}'.format(self.CopleySerial.portstr)
        print('CopleySerial Info: '+msg)

    def debug_stream(self, *msg):
        '''
            output debug info
            [issue]:
            有待完善
        '''
        #----- PROTECTED REGION ID(InitialDeviceClass.debug_stream) ENABLED START -----#
        strs = ''
        for target_tuple in reversed(msg):
            strs = str(target_tuple) + '\t' + strs
        print(strs)
        #----- PROTECTED REGION END -----#	//	CopleyControl.debug_stream

    def OutputResult(self, num=15):
        while num:
            subprocess.call("clear")
            # self.debug_stream('\n# Temperature')
            self.RxD('g r0x20\r') # Drive Temperature A/D Reading. Units: degrees C.

            # self.debug_stream('\n# Voltage')
            self.RxD('g r0x38 \r') # Actual Motor Current. Units: 0.01 A. D\Q
            self.RxD('g r0x0c\r') # Actual Current, Q axis of rotor space. Units: 0.01 A.
            self.RxD('g r0x1E\r') # High Voltage A/D Reading. Units: 100 mV.

            # self.debug_stream('\n# Velocity')
            self.RxD('g r0xcb\r') # Trajectory Maximum Velocity. Units: 0.1 counts/s.
            self.RxD('g r0xcc\r') # Maximum acceleration rate(0xcc). Units: 10 counts/second2.

            self.RxD('g r0x3d \r') # Trajectory Destination Position. Units: encoder counts.

            # self.debug_stream('\n# Position')
            self.RxD('g r0x2d \r') # R*_Commanded Position. Units: counts. / 位置模式下设置编码器的位置
            self.RxD('g r0x32\r') # Motor position. Units: counts. 
            # self.RxD('s r0x32 0\r')
            self.RxD('g r0x17\r') # Motor position. Units: counts.
            # self.RxD('s r0x17 0\r')

            # self.debug_stream('\n# Homing')
            # c2 = self.RxD('g r0xc2\r')
            # c9 = self.RxD('g r0xc9\r')
            # bit_c2 = BitsMapped_HM('0xC2', c2[2:])
            # bit_c9 = BitsMapped_HM('0xC9', c9[2:])
            # print('C2: {}\nC9: {}'.format(bit_c2, bit_c9))

            print('--- --- ---')
            time.sleep(0.1)
            num -= 1

ports = Port() # 创建Port()类

# Current Mode

In [3]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

def current_mode_fun(nodeid=0):
    ports.NodeID = nodeid
    # 在编程的当前模式下启用驱动器。在0.5秒内将输出电流斜坡上升至2A。控制器监视输出电流，达到2 A后，电流将在2秒钟内下降到1A。
    # ports.RxD('s r0x2f 349546\r') # 58709 # 15333 # 15333 # 1rpm # 6048585/5511281
    ports.RxD('s r0x24 0\r') # 1
    ports.RxD('s r0x6a 100\r') # Current ramp limit. Units: mA/second./数值越大，响应越快；即设置大一点好
    ports.RxD('s r0x02 200\r') # Programmed current value. Units: 0.01 A. 需要时刻调整的阈值(根据编码器的数值)/负载>0x02则正转；反之反转；相等不转
    # ports.RxD('s r0x24 1\r') # 1

    # ports.RxD('s r0x6a 500\r') # 将新的斜坡速率设置为0.5 A/s。
    # ports.RxD('s r0x02 100\r') # 将输出电平更改为1A。输出电流将以0.5 A/s的速度开始减小。
    # ports.RxD('g r0x0c\r') # 读取驱动器的实际电流输出。 示例显示返回的值等于1.50A。
    # ports.RxD('s r0x24 0\r') # 禁用驱动器。

    # ports.RxD('g r0x38 \r') # Actual Motor Current. Units: 0.01 A.
    # ports.RxD('g r0x03\r') # Winding A Current. Units: 0.01 A. Actual current measured at winding A.
    # ports.RxD('g r0x04\r') # Winding B Current. Units: 0.01 A. Actual current measured at winding B.
    # ports.RxD('g r0x0b\r') # Actual Current, D axis of rotor space. Units: 0.01 A.
    # ports.RxD('g r0x0c\r') # Actual Current, Q axis of rotor space. Units: 0.01 A.
    ports.RxD('g r0x21\r') # Peak Current Limit. Units: 0.01 A./693
    ports.RxD('g r0x22\r') # Continuous Current Limit. Units: 0.01 A./347
    ports.RxD('g r0x23\r') # Time at Peak Current Limit. Units: ms./1000
    ports.RxD('g r0x25\r') # Limited Motor Current Command. Units: 0.01 A./15

current_mode_fun(nodeid=0)
current_mode_fun(nodeid=1)
# ports.NodeID = 0
ports.OutputResult(num=0)

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True
RxD: 0 s r0x24 0	ok
RxD: 0 s r0x6a 100	ok
RxD: 0 s r0x02 200	ok
RxD: 0 g r0x21	v 693
RxD: 0 g r0x22	v 347
RxD: 0 g r0x23	v 1000
RxD: 0 g r0x25	v 200
RxD: 1 s r0x24 0	ok
RxD: 1 s r0x6a 100	ok
RxD: 1 s r0x02 200	ok
RxD: 1 g r0x21	v 693
RxD: 1 g r0x22	v 347
RxD: 1 g r0x23	v 1000
RxD: 1 g r0x25	v 200


# Speed Mode

In [10]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

def speed_mode_fun(nodeid=0):
    ports.NodeID = nodeid
    # ports.RxD('s r0x36 1000\r')
    # ports.RxD('s r0x37 1000\r')
    # ports.RxD('s r0x2f 849546\r') # 549546/274773
    # ports.RxD('s r0x24 11\r')
    ports.RxD('s r0x24 0\r')

    # ports.RxD('g r0x18\r') # Actual Velocity. Units: 0.1 encoder counts/s.
    # ports.RxD('g r0x2d \r') # R*_Commanded Position. Units: counts. / 位置模式下设置编码器的位置(实际位置)
    # ports.RxD('g r0x32\r') # Motor position. Units: counts. # ports.RxD('s r0x32 0\r')
    # ports.RxD('g r0x17\r') # Motor position. Units: counts. # ports.RxD('s r0x17 0\r')
    # ports.RxD('g r0x3a\r') # Velocity Loop Velocity Limit. Units 0.1 counts/s./5495467

    # Software Limit
    # ports.RxD('s r0xb8 100\r')
    # ports.RxD('s r0xb9 -100\r')

speed_mode_fun(nodeid=0)
speed_mode_fun(nodeid=1)
# ports.OutputResult(num=1)

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True
RxD: 0 s r0x24 0	ok
RxD: 1 s r0x24 0	ok


# Position Mode

In [3]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

def position_mode_fun(nodeid):
    ports.NodeID = nodeid
    ports.RxD('s r0x24 21\r') # Desired State, Bits: 21 伺服模式下，位置环由轨迹发生器驱动
    ports.RxD('s r0xc8 256\r') # Give trajectory profile mode(0xc8)./在0、256模式下可以不用停止直接发送t 1就更改参数(记得关掉0x24)
        # * 0     /0 0000 0000 = Absolute move, trapezoidal profile.
        # 1     /0 0000 0001 = Absolute move, S-curve profile.
        # * 256/1 0000 0000 = If set, relative move, trapezoidal profile.
        # 257/1 0000 0001 = If clear, relative move, S-curve profile.
        # * 2    / 0 0000 0010 = Velocity move.
    ports.RxD('s r0xca 512000\r') # Trajectory Generator Position Command(0xca). Units: Counts. 1024*4*86=352256/1000*4*23=92000/1000*4*128=512000
    # 117555 = 1000*4*23 * (46/360*10)对于当前右侧，正向是推出
    # 27662正值是伸出
    ports.RxD('s r0xcb 549546\r') # Trajectory Maximum Velocity. Units: 0.1 counts/s. # 8050*1024*4*10/60=5495466.666.../5495467(0x3a)
    ports.RxD('s r0xcc 100000\r') # Trajectory Maximum Acceleration. Units: 10 counts/s2. # 1000000差不多就是极限了
    ports.RxD('s r0xcd 100000\r') # Trajectory Maximum Deceleration. Units: 10 counts/s2.

    # ports.RxD('s r0xb8 352256\r')
    # ports.RxD('s r0xb9 -352256\r')
    # ports.RxD('g r0xb8\r') # 1024*4*86=352256，默认值是一圈？并且断电后会重置
    # ports.RxD('g r0xb9\r') # 1024*4*86=352256，默认值是一圈？
    # Position
    # ports.RxD('g r0x3d \r') # Trajectory Destination Position. Units: encoder counts. / 这个是0xca设定的值
    # ports.RxD('g r0x2d \r') # R*_Commanded Position. Units: counts. / 位置模式下设置编码器的位置(实际位置)，若是速度模式就变成了0xca设定的值
    # ports.RxD('g r0x17\r') # 获取编码器的实际位置，区别于0x2d
    ports.RxD('g r0x32\r') # Motor position. Units: counts. / 实际位置

    ports.RxD('t 1\r')
    # ports.RxD('t 0\r')

position_mode_fun(nodeid=0)
position_mode_fun(nodeid=1)
ports.OutputResult(num=3)
# 在位置模式下的速度模式和位置模式进行切换时候，电机可以自然切换，不会突然停顿，这也是位置模式下速度模式存在的意义
# 例如，在路径规划中，可能有的时候需要控制位置，又有时候切换成速度方便
# 这种，放在机器人自己运动的时候更好

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True
RxD: 0 s r0xc8 256	ok
RxD: 0 s r0xca 512000	ok
RxD: 0 s r0xcb 549546	ok
RxD: 0 s r0xcc 100000	ok
RxD: 0 s r0xcd 100000	ok
RxD: 0 g r0x32	v 512000
RxD: 0 t 1	ok
RxD: 1 s r0xc8 256	ok
RxD: 1 s r0xca 512000	ok
RxD: 1 s r0xcb 549546	ok
RxD: 1 s r0xcc 100000	ok
RxD: 1 s r0xcd 100000	ok
RxD: 1 g r0x32	v 487112
RxD: 1 t 1	ok
RxD: 1 g r0x20	v 39
RxD: 1 g r0x38 	v -11
RxD: 1 g r0x0c	v -10
RxD: 1 g r0x1E	v 245
RxD: 1 g r0xcb	v 549546
RxD: 1 g r0xcc	v 100000
RxD: 1 g r0x3d 	v 999112
RxD: 1 g r0x2d 	v 506979
RxD: 1 g r0x32	v 509563
RxD: 1 g r0x17	v 512203
--- --- ---
RxD: 1 g r0x20	v 39
RxD: 1 g r0x38 	v -7
RxD: 1 g r0x0c	v -8
RxD: 1 g r0x1E	v 245
RxD: 1 g r0xcb	v 549546
RxD: 1 g r0xcc	v 100000
RxD: 1 g r0x3d 	v 999112
RxD: 1 g r0x2d 	v 539549
RxD: 1 g r0x32	v 542132
RxD: 1 g r0x17	v 544751
--- --- ---
RxD: 1 g r0x20	v 39
RxD: 1 g r0x38 	v -4
RxD: 1 g r0x0c	v -9
RxD: 1 g r0x1E	v 245
RxD: 1 g r0xcb	v 54954

# Homing Method

In [395]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

def home_mode_fun(nodeid):
    ports.NodeID = nodeid
    ports.RxD('s r0x24 21\r') # Desired State / Position
    # ports.RxD('s r0xc8 0\r') # Trajectory Profile Mode.

    ports.RxD('s r0xc2 544\r') # Homing Method Configuration./512
    # ports.RxD('s r0xc3 4495467\r') # Homing Velocity (fast moves)(0xc3). Units: 0.1 counts/s.
    # ports.RxD('s r0xc4 149546\r') # Homing Velocity (slow moves)(0xc4). Units: 0.1 counts/s.
    # ports.RxD('s r0xc5 10000\r') # Homing Acceleration/Deceleration(0xc5). Units: 10 counts/s2.
    # ports.RxD('s r0xc6 1000\r') # Home Offset(0xc6). Units: counts. 
    # ports.RxD('s r0xc7 10\r') # Homing Current Limit(0xc7). Units: 0.01 A.
    # ports.RxD('s r0xbf 100\r') # Home to Hard Stop Delay Time. Units: ms.

    # ports.RxD('g r0xc2\r') # Homing Method Configuration./512
    # ports.RxD('g r0xc9\r') # The trajectory register parameter (0xc9) provides trajectory generator status information.
    # ports.RxD('g r0xbf\r') # Home to Hard Stop Delay Time. Units: ms./250
    # ports.RxD('g r0xc6\r') # Home Offset(0xc6). Units: counts. /0
    # ports.RxD('g r0xc7\r') # Homing Current Limit(0xc7). Units: 0.01 A.

    # Software Limit
    # ports.RxD('s r0xb8 352256\r') # Positive Software Limit value(0xb8). Units: counts.
    # ports.RxD('s r0xb9 -352256\r') # Negative Software Limit(0xb9). Units: counts. 
    # ports.RxD('g r0xb8\r') # Positive Software Limit value(0xb8). Units: counts.
    # ports.RxD('g r0xb9\r') # Negative Software Limit(0xb9). Units: counts. 

    ports.RxD('t 2\r')
    # ports.RxD('t 0\r')
    print('~~~~~~~~~~~~~~~~~~')

home_mode_fun(nodeid=0)
home_mode_fun(nodeid=1)
# ports.OutputResult()
# [Note]:
# 所谓的归位模式，本质上就是一种特殊的位置模式

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True
RxD: 0 s r0x24 21	ok
RxD: 0 s r0xc2 544	ok
RxD: 0 t 2	ok
~~~~~~~~~~~~~~~~~~
RxD: 1 s r0x24 21	ok
RxD: 1 s r0xc2 544	ok
RxD: 1 t 2	ok
~~~~~~~~~~~~~~~~~~


# Feedback

In [360]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

def feedback_fun(nodeid=0):
    ports.NodeID = nodeid
    # ports.RxD('enc clear\r')

    ports.debug_stream('\n# Temperature')
    ports.RxD('g r0x20\r') # Drive Temperature A/D Reading. Units: degrees C.

    # ports.RxD('g r0xcb\r') # Trajectory Maximum Velocity. Units: 0.1 counts/s.
    # ports.RxD('g r0xcc\r') # Maximum acceleration rate(0xcc). Units: 10 counts/second2.

    # ports.debug_stream('\n# Voltage')
    # ports.RxD('g r0x38 \r') # Actual Motor Current. Units: 0.01 A.
    # ports.RxD('g r0x1E\r') # High Voltage A/D Reading. Units: 100 mV.

    ports.debug_stream('\n# Speed')
    ports.RxD('g r0x18 \r') # Actual Velocity. Units: 0.1 encoder counts/s.

    ports.debug_stream('\n# Position')
    # ports.RxD('g r0x2d \r') # R*_Commanded Position. Units: counts. / 位置模式下设置编码器的位置
    ports.RxD('g r0x3d \r') # Trajectory Destination Position. Units: encoder counts.
    # ports.RxD('s r0x32 0\r')
    ports.RxD('g r0x32\r') # Motor position. Units: counts. 
    # ports.RxD('g r0x17\r') # Motor position. Units: counts.
    # ports.RxD('s r0x17 0\r')
    # ports.RxD('g r0x3a\r') # Velocity Loop Velocity Limit. Units 0.1 counts/s./5495467
feedback_fun(nodeid=0)
feedback_fun(nodeid=1)

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True

# Temperature	
RxD: 0 g r0x20	v 42

# Speed	
RxD: 0 g r0x18 	v 0

# Position	
RxD: 0 g r0x3d 	v 0
RxD: 0 g r0x32	v -1

# Temperature	
RxD: 1 g r0x20	v 44

# Speed	
RxD: 1 g r0x18 	v 0

# Position	
RxD: 1 g r0x3d 	v 0
RxD: 1 g r0x32	v 0


# Baud

In [5]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

ports.RxD('g r0x90\r') # Serial Port Baud Rate. Units: bits/s.
# CAN
ports.RxD('g r0xc0\r') # CAN Network Node ID.
ports.RxD('g r0xc1\r') # CAN Network Node ID Configuration./# 20480(0101 0000 0000 0000) = 50,000

RxD: g r0x90	v 9615
RxD: g r0xc0	v 0
RxD: g r0xc1	v 20480
CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True


# Event Status Register
Event Status Register(0xA0). Bits: 0~31

In [3]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态
from copley_lib.ParamDict_HomingMethod import BitsMapped as BitsMapped_HM
from copley_lib.ParamDict_EventStatus import BitsMapped as BitsMapped_ES


def event_status_register(nodeid=0):
    ports.NodeID = nodeid
    ports.RxD('s r0xa4 0xffff\r')
    ports.RxD('s r0xa1 0xffffffff\r')
    a0 = ports.RxD('g r0xa0\r')
    a1 = ports.RxD('g r0xa1\r')
    a4 = ports.RxD('g r0xa4\r')
    a7 = ports.RxD('g r0xa7\r') # 5119/0001,0011,1111,1111
    bit_a0 = BitsMapped_ES('0xA0', a0[2:])
    bit_a1 = BitsMapped_ES('0xA0', a1[2:])
    bit_a4 = BitsMapped_ES('0xA4', a4[2:])
    print('A0: \n{}\nA1: \n{}\nA4: \n{}\n'.format(bit_a0, bit_a1, bit_a4))
    print('~~~~~~~~~~~~~~~~~~~')

    # c2 = ports.RxD('g r0xc2\r')
    # c9 = ports.RxD('g r0xc9\r')
    # bit_c2 = BitsMapped_HM('0xC2', c2[2:])
    # bit_c9 = BitsMapped_HM('0xC9', c9[2:])
    # print('C2: \n{}\nC9: \n{}\n'.format(bit_c2, bit_c9))
    # print('++++++++++++++++++++++')

event_status_register(nodeid=0)
event_status_register(nodeid=1)

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True


TypeError: unicode strings are not supported, please encode to bytes: '0 s r0xa4 0xffff\r'

# Direction

In [29]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

# 这两个参数要同时改变
# 若只改变0x4e电机转向确实会改变但是因为编码器的方向没有改变（编码器没有起作用，所以电机会全速运动）
# 同理，若只改变编码器的方向，电机的转向未变，也是出问题
ports.RxD_Line('0 g f0x4e\r')
ports.RxD_Line('1 g f0x4e\r')
# ports.RxD_Line('0 s f0x4e 0\r')
# ports.RxD_Line('1 s f0x4e 0\r')

ports.RxD_Line('0 g f0x65\r')
ports.RxD_Line('1 g f0x65\r')
ports.RxD_Line('0 s f0x65 0\r')
ports.RxD_Line('1 s f0x65 1\r')

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True
RxD: 0 g f0x4e	v 0
RxD: 1 g f0x4e	v 0
RxD: 0 g f0x65	v 0
RxD: 1 g f0x65	v 0
RxD: 0 s f0x65 0	ok
RxD: 1 s f0x65 1	ok


'ok'

# 多点CAN

In [32]:
ports = Port() # 创建Port()类
ports.PortInfo(mode='full_msg') # 输出当前端口状态

ports.RxD_Line('0 g r0xc1\r')
ports.RxD_Line('1 g r0xc1\r')

CopleySerial Info: 
	- PortID: /dev/UCR_Drive
	- Buad: 9600
	- isOpen: True
	- Readable: True
RxD: 0 g r0xc1	v 20480
RxD: 1 g r0xc1	v 20481


'v 20481'

# 有关波特率的计算
在力矩模式下，是根据位置调整电流阈值，进一步控制方向，从而反向调整位置，也就是最终想要调整的是位置
在位置模式下，是根据电流调整位置，进一步控制位置，从而反向调节电流，也就是最终想要调整的是电流
也就是说，如果想要调整的是位置，那么就要采用电流模式！！！
退而求其次，也可以只是控制位置，不管电流，只要电流安全就行。
但是换句话说，按照力矩模式来，最终调整的是位置的话，这不就是位置模式吗？
区别在于，这里除了编码器的位置还引入了力矩，即两个参量来确定位置

试了一下循环读取两个驱动的数据，分别为9600/115200
一次循环是10条指令，每个驱动器5条 # 90s内有大概275次循环/382
一次循环是2条指令，每个驱动器1条 # 90s内大概1433次循环/1914
两次大概可以看出来，总量接近都是2800条指令，即32毫秒一条指令

一次循环是10条指令，只是第一个驱动器 # 90s内大概332次循环/575
一次循环是2条指令，只第一个驱动器 # 90s内大概1818次循环/2861

In [ ]:
self.RxD('0 g r0x18\r') # Actual Velocity. Units: 0.1 encoder counts/s.
self.RxD('0 g r0x17\r') # Actual Position. Units: Counts.
self.RxD('0 g r0x02\r') # Current loop programmed value. Units: 0.01 A.
self.RxD('0 g r0x38 \r') # Actual Motor Current. Units: 0.01 A.
self.RxD('0 g r0x0c\r') # Actual Current, Q axis of rotor space. Units: 0.01 A.

self.RxD('1 g r0x18\r') # Actual Velocity. Units: 0.1 encoder counts/s.
self.RxD('1 g r0x17\r') # Actual Position. Units: Counts.
self.RxD('1 g r0x02\r') # Current loop programmed value. Units: 0.01 A.
self.RxD('1 g r0x38 \r') # Actual Motor Current. Units: 0.01 A.
self.RxD('1 g r0x0c\r') # Actual Current, Q axis of rotor space. Units: 0.01 A.

# 有关电机启动顺序
考虑用多线程的方式调用两个电机的实例，达到电机同时转动
尝试使用电机驱动
整理算法
做界面
保存数据
确定电机转动方向
位置模式需要增加参数，即限位
调整波特率的函数需要调试